## Calculate Raw image correction paramters
*Author: Haiyun Huang 2025 with Deepseek*

This document aims to acquire mathmatically accurate correction parameters for raw images.

In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
import mvsdk
import cv2
import numpy as np
import matplotlib.pyplot as plt

from huateng_camera_v2_tc_mod import Camera
from raw_processing import raw_awb, raw_wb
from correction_utils import suppress_output

In [3]:
EXPOSURE_TIME = 10 # ms
ADC_MAX_LEVEL = 2**12 - 1 # No greater than 65535

### 01. Acquiring AWB Correction Parameters
The WB process will do:
- (optional) extra dBLC, 
- clipping to per channel max level, 
- scales the RAW channels to 16bit.

Acquiring params including:
- (optional) RGB dBLC (after BLC), 
- RGB gains.

In [4]:
# Acquire preview image
DevList = mvsdk.CameraEnumerateDevice()
mycam = Camera(DevList[0], EXPOSURE_TIME, gain=1, hibitdepth=1)

mycam.open()
raw_img = mycam.grab_raw()
mycam.close()
# img最大值最小值
print(f'RAW Grabbed: img.max(): {raw_img.max()}, img.min(): {raw_img.min()}')
plt.imshow(raw_img)
plt.title('RAW Preview')
plt.show()

Camera.open: Camera supports 2 pixel format(s)
Camera.open: Pixel format 0: Bayer BG 8bit (1Bpp)
Camera.open: Pixel format 1: Bayer BG 12bit Packed (1.5Bpp)
Camera.open: Using 12bit pixel format.
Camera.open: Timecode disabled. Allocating original image buffer size: 30081024 bytes.
Camera.open: Raw start bit = -1
RAW Grabbed: img.max(): 4094, img.min(): 32


In [5]:
# Setting essential params for acquiring correction params
repeats = 100
BLC = 32
BAYER_PATTERN = 'BGGR'

# 白平衡
# 定义两个ROI用于自动白平衡
# ROI 1 (亮区)
x_slice_1 = slice(1440, 1500)
y_slice_1 = slice(920, 980)
awb_roi_1 = (y_slice_1, x_slice_1) # y,x

# ROI 2 (暗区)
x_slice_2 = slice(1440, 1500)
y_slice_2 = slice(1120, 1180)
awb_roi_2 = (y_slice_2, x_slice_2) # y,x

In [6]:
# Acquire images

DevList = mvsdk.CameraEnumerateDevice()
mycam = Camera(DevList[0], EXPOSURE_TIME, gain=1, hibitdepth=1)
mycam.open()

imgs = []
for i in range(repeats):
    img = mycam.grab_raw()
    imgs.append(img)
mycam.close()

Camera.open: Camera supports 2 pixel format(s)
Camera.open: Pixel format 0: Bayer BG 8bit (1Bpp)
Camera.open: Pixel format 1: Bayer BG 12bit Packed (1.5Bpp)
Camera.open: Using 12bit pixel format.
Camera.open: Timecode disabled. Allocating original image buffer size: 30081024 bytes.
Camera.open: Raw start bit = -1


In [7]:
# Calculate avg. correction parameters
raw_awb_slience = suppress_output(raw_awb)

awb_params_list = []
for img in imgs:
    # basic BLC
    img_BLC = img - BLC
    # AWB
    _, awb_params = raw_awb_slience(img_BLC, awb_roi_1, awb_roi_2, pattern=BAYER_PATTERN, clip_max_level=ADC_MAX_LEVEL-BLC)
    awb_params_list.append(awb_params)
awb_params_np = np.array(awb_params_list)
awb_params_avg = np.mean(awb_params_np, axis=0)
awb_params_se = np.std(awb_params_np, ddof=1, axis=0)/np.sqrt(repeats)

print('Average AWB params: (mean ± SE)')
print(f'R gain: {awb_params_avg[0]:.4f} ± {awb_params_se[0]:.4f}')
print(f'G gain: {awb_params_avg[1]:.4f} ± {awb_params_se[1]:.4f}')
print(f'B gain: {awb_params_avg[2]:.4f} ± {awb_params_se[2]:.4f}')
print(f'R dBLC: {awb_params_avg[3]:.4f} ± {awb_params_se[3]:.4f}')
print(f'G dBLC: {awb_params_avg[4]:.4f} ± {awb_params_se[4]:.4f}')
print(f'B dBLC: {awb_params_avg[5]:.4f} ± {awb_params_se[5]:.4f}')
print(f'Copy format: ({', '.join(map(str, awb_params_avg))})')
awb_params = tuple(awb_params_avg)
print('Parameters saved in variable `awb_params`')


Average AWB params: (mean ± SE)
R gain: 1.8539 ± 0.0003
G gain: 1.2618 ± 0.0002
B gain: 1.0000 ± 0.0000
R dBLC: -14.4534 ± 0.1373
G dBLC: -11.8186 ± 0.1712
B dBLC: 0.0000 ± 0.0000
Copy format: (1.85387511102, 1.26180502111, 1.0, -14.4534427761, -11.8186137314, 0.0)
Parameters saved in variable `awb_params`


#### Results

D65 Room LED Light
> Average AWB params: (mean ± SE)\
`R gain`: 1.8722 ± 0.0005\
`G gain`: 1.2736 ± 0.0002\
`B gain`: 1.0000 ± 0.0000\
`R dBLC`: -16.2625 ± 0.2017\
`G dBLC`: -13.0992 ± 0.2143\
`B dBLC`: 0.0000 ± 0.0000\
Copy format: `(1.87217887201, 1.27358336204, 1.0, -16.2625453031, -13.099179932, 0.0)`

D50 Sunlike Light (10ms exposure, white patch ~ 3000-4000 DN @ 12bit)
> Average AWB params: (mean ± SE)\
`R gain`: 1.6223 ± 0.0003\
`G gain`: 1.3053 ± 0.0002\
`B gain`: 1.0000 ± 0.0000\
`R dBLC`: -5.5832 ± 0.1601\
`G dBLC`: -6.5196 ± 0.1868\
`B dBLC`: 0.0000 ± 0.0000\
Copy format: `(1.62229078895, 1.30534488103, 1.0, -5.58319935118, -6.51955880128, 0.0)`\
Parameters saved in variable `awb_params`


### 02. Calculate Forward Matrix
Transform colors from device $RGB$ to $XYZ_{D50}$

#### a1. Preparing ColorChecker24 Reference Data
Convert reference $Lab_{D50}$ data to $XYZ$ for given illuminant by *Bradford* + *von Keris* method.

The reference ColorChecker24 patches are measured under D50 illuminant in $Lab_{D50}$ format\
We need to transform such refernece $Lab$ value to $XYZ$ space under the constraint of D50 illuminant.

The layout of CC24: 4 rows, 6 columns. `A1` is at upper left, `F4` is at lower right.

多解释一些：
1. 对于反射式测试卡，反射率是其本质，而对于传感器，光谱辐照是其本质。
2. D50照明定义了照明谱，只有在给定照明谱的情况下，才能拿到光谱辐照，拿到加色性的XYZ值。
3. 这也是为什么测试卡用Lab标注值，而不是XYZ，因为XYZ会随着照明不同而改变。但Lab是归一化到白点（*相对*稳定，此为von Kries假设下的方法），并且定义了测量白点是D50（Lab空间需要给定白点才在辐照水平有意义）。
4. $XYZ_{D50}$的下标并没有实际意义，因为XYZ空间不依赖白点而存在，只是为了表示是D50照明下获得的值。
5. 对于不同白点的转换，常用Bradford比色法，主要是将XYZ值转移到LMS空间后缩放。不同比色法定义的这个转移矩阵$M_A$不同。
6. 对于更精确的应用，应当直接从各色块的反射率曲线推导给定光源（如D65）下的XYZ值，而不是用$Lab_{D50}$通过Bradford。 参见：https://babelcolor.com/colorchecker-2.htm#xl_CCP2_images
7. 由于新版ColorChecker24没有平均的反射率曲线可用，本文档推荐使用Bradford比色法处理照明非D50的情况。

The basic idea is, transforming $XYZ$ to physiologically relavent $LMS$ space, scaling in $LMS$ space (based on *von Kries* hypothesis), and then transform back.\
There are multiple options of matirx for $XYZ$ to $LMS$, probably because of different illuminant spectra,\
including *Identical*, *Bradford* methods, etc.\
See documents of `colour.adaptation.CAT_*` for more methods based on *von Kries* hypothesis.

See more mathmatical details:
  1. https://fujiwaratko.sakura.ne.jp/infosci/colorspace/bradford_e.html\
  2. http://www.brucelindbloom.com/index.html?Eqn_ChromAdapt.html\

For information about XYZ-Lab transformation, see: 
  1. https://fujiwaratko.sakura.ne.jp/infosci/colorspace/colorspace3_e.html\

In [8]:
# Specify the illuminant of the calibration images, CIE 1931 XYZ value.

import colour

illuminant = colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']

In [9]:
# Loading ColorChecker24 Lab_D50 value from X-rite data(after 2015)
import numpy as np

def parse_data_file(file_path):
    # Deepseek
    # 初始化存储变量
    string_list = []
    numeric_data = []
    
    # 标志是否开始/结束读取数据
    reading_data = False
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # 去除首尾空白字符
            
            if not reading_data:
                if line == 'BEGIN_DATA':
                    reading_data = True
                continue
            else:
                if line == 'END_DATA':
                    break
                
                # 分割数据行
                parts = line.split('\t')
                if len(parts) != 4:
                    continue  # 跳过格式不正确的行
                
                # 解析数据
                string_part = parts[0]
                try:
                    numeric_parts = [float(x) for x in parts[1:4]]
                except ValueError:
                    continue  # 跳过数值转换失败的行
                
                # 存储数据
                string_list.append(string_part)
                numeric_data.append(numeric_parts)
    
    # 将数值数据转换为N×3的NumPy数组
    numeric_array = np.array(numeric_data)
    
    return string_list, numeric_array

patch_name, lab_values = parse_data_file('RefPatches.txt')
# The value is column first, where A* is the first column.
patch_name_row_first = np.array(patch_name).reshape(4, 6, order='F').ravel(order='C')
lab_values_row_first = lab_values.reshape((4, 6, 3), order='F').reshape((-1, 3), order='C')
print(patch_name_row_first)
print(lab_values_row_first)


['A1' 'B1' 'C1' 'D1' 'E1' 'F1' 'A2' 'B2' 'C2' 'D2' 'E2' 'F2' 'A3' 'B3' 'C3'
 'D3' 'E3' 'F3' 'A4' 'B4' 'C4' 'D4' 'E4' 'F4']
[[  3.75400000e+01   1.43700000e+01   1.49200000e+01]
 [  6.46600000e+01   1.92700000e+01   1.75000000e+01]
 [  4.93200000e+01  -3.82000000e+00  -2.25400000e+01]
 [  4.34600000e+01  -1.27400000e+01   2.27200000e+01]
 [  5.49400000e+01   9.61000000e+00  -2.47900000e+01]
 [  7.04800000e+01  -3.22600000e+01  -3.70000000e-01]
 [  6.27300000e+01   3.58300000e+01   5.65000000e+01]
 [  3.94300000e+01   1.07500000e+01  -4.51700000e+01]
 [  5.05700000e+01   4.86400000e+01   1.66700000e+01]
 [  3.01000000e+01   2.25400000e+01  -2.08700000e+01]
 [  7.17700000e+01  -2.41300000e+01   5.81900000e+01]
 [  7.15100000e+01   1.82400000e+01   6.73700000e+01]
 [  2.83700000e+01   1.54200000e+01  -4.98000000e+01]
 [  5.43800000e+01  -3.97200000e+01   3.22700000e+01]
 [  4.24300000e+01   5.10500000e+01   2.86200000e+01]
 [  8.18000000e+01   2.67000000e+00   8.04100000e+01]
 [  5.0630000

In [10]:
# Transform Lab_D50 to XYZ_D50 using colour-science package

XYZ_D50_values = colour.Lab_to_XYZ(lab_values_row_first, 
                                   illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']) # Lab measured in D50
# The D50 subscript indicates that the XYZ values are obtained under the D50 illuminant.
# print(XYZ_D50_values)
print(colour.XYZ_to_xy(XYZ_D50_values[18])) # the brightest white patch

[ 0.34891221  0.36417406]


The XYZ values could be transform to other illuminant by:
  1. Implicitly using *Identical* matrix (NOT recommanded) for XYZ to LMS transform, then scaling under *von Kries* hypothesis.\
     This can be done by changing the `illuminant` parameter in `Lab_to_XYZ` function.\
     Also see the document and the source code of `Lab_to_XYZ` function. (https://colour.readthedocs.io/en/develop/_modules/colour/models/cie_lab.html#Lab_to_XYZ)\

  2. Explicitly using *Bradford* matrix or other matrix (such as *CAT02*) to transform.

The area prefers *Bradford* method (For historical reason? maybe have a look for *CAT02* matrix), but it would be better if we can **measure the spectra** for the chart.

We have generated $XYZ_{D65}$ values by all three methods below for the CC24 chart.

In [11]:
# TODO: Manually written Lab to XYZ_D50 transformation

def lab_to_xyz_d50(lab_values):
    # The basic idea is, transforming XYZ to physiologically relavent LMS space, scaling in LMS space, and then transform back.
    # There are multiple options of matirx for XYZ to LMS, probably because of different illuminant spectra.
    # including Identical, Bradford and Von Kries
    # See also: 
    #   1. https://fujiwaratko.sakura.ne.jp/infosci/colorspace/bradford_e.html
    #   2. http://www.brucelindbloom.com/index.html?Eqn_ChromAdapt.html
    # For information about XYZ-Lab transformation, see: 
    #   1. https://fujiwaratko.sakura.ne.jp/infosci/colorspace/colorspace3_e.html
    
    # Using Bradford M_A matrix for XYZ-LMS conversion
    M_A = np.array([
        [0.8951, 0.2664, -0.1614],
        [-0.7502, 1.7135, 0.0367],
        [0.0389, -0.0685, 1.0296]
    ])
    # 定义D50white point
    D50 = np.array([96.42, 100.0, 82.52])

In [12]:
# Transform XYZ (D50 illuminant) to XYZ (D65 illuminant)

# Method 1. Change illuminant when Lab->XYZ (implicitly)
XYZ_D65_values_identical = colour.Lab_to_XYZ(lab_values_row_first, illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65'])

# Method 2. Change illuminant under XYZ using Bradford
color_correction_mtx_bradford = colour.adaptation.matrix_chromatic_adaptation_VonKries(
    XYZ_w=colour.xyY_to_XYZ(colour.xy_to_xyY(colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50'])),
    XYZ_wr=colour.xyY_to_XYZ(colour.xy_to_xyY(colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65'])),
    transform='Bradford',
)
# Transform from w to wr, the naming is arbitrary.
# This produce M_{cat} = M_A^{-1} @ S @M_A, where M_A is Bradford matrix and S is scaling matrix depending on src. and dest. illuminant.
XYZ_D65_values_bradford = XYZ_D50_values @ color_correction_mtx_bradford.T

# Method 3. Change illuminant under XYZ using CAT02
color_correction_mtx_cat02 = colour.adaptation.matrix_chromatic_adaptation_VonKries(
    XYZ_w=colour.xyY_to_XYZ(colour.xy_to_xyY(colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50'])),
    XYZ_wr=colour.xyY_to_XYZ(colour.xy_to_xyY(colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65'])),
    transform='CAT02',
)
XYZ_D65_values_cat02 = XYZ_D50_values @ color_correction_mtx_cat02.T

# print(XYZ_D65_values)
print('White patch(0.05D):')
print(f'Identical xy: {colour.XYZ_to_xy(XYZ_D65_values_identical[18])}')
print(f'Bradford xy: {colour.XYZ_to_xy(XYZ_D65_values_bradford[18])}') # the brightest white patch
print(f'CAT02 xy: {colour.XYZ_to_xy(XYZ_D65_values_cat02[18])}')
# Here we could see the difference by using Bradford and Identical matrix in von Keris adaptation method.

White patch(0.05D):
Identical xy: [ 0.31644451  0.33509554]
Bradford xy: [ 0.31606103  0.33525624]
CAT02 xy: [ 0.31598245  0.33513653]


#### a2. Observation
For dE calculation, we need Lab values again, which requires XYZ -> Lab transformation with illuminant again.\
If the XYZ->Lab inverse path is not the same with the forward one, dE increases prominantly.

For the three method mentioned above, we should **carefully calculate back** to Lab in order to get the accurate dE,\
which reflects the actual residual error introduced by the color correction process.

In [13]:
# The illuminant transformation causes dE:
# LabD50 -> XYZD50 -Bradford-> XYZD65 -(Implicitly Identical) -> LabD50
dE_demo_1 = colour.delta_E(lab_values_row_first, colour.XYZ_to_Lab(XYZ_D65_values_bradford, illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65'])).reshape(4,6)
print(dE_demo_1)
print(f'dE max: {np.max(dE_demo_1):.2f}, min: {np.min(dE_demo_1):.2f}, avg: {np.mean(dE_demo_1):.2f}')

# It seems like you should do all things inversely to get near 0 dE.
dE_demo_2 = colour.delta_E(lab_values_row_first, colour.XYZ_to_Lab(XYZ_D65_values_identical, illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65'])).reshape(4,6)
print(f'dE max: {np.max(dE_demo_2):.2f}, min: {np.min(dE_demo_2):.2f}, avg: {np.mean(dE_demo_2):.2f}')

dE_demo_3 = colour.delta_E(lab_values_row_first, 
                           colour.XYZ_to_Lab(XYZ_D65_values_bradford @ np.linalg.inv(color_correction_mtx_bradford).T, 
                                             illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']
                                             )
                           ).reshape(4,6)
print(f'dE max: {np.max(dE_demo_3):.2f}, min: {np.min(dE_demo_3):.2f}, avg: {np.mean(dE_demo_3):.2f}')


[[ 0.76300704  0.78461623  2.55468098  1.27930158  2.1124803   0.36321578]
 [ 1.27623702  4.2126116   0.74909598  1.02017003  1.98745382  2.14843468]
 [ 4.65064181  1.03465616  0.78876897  2.62805336  0.48428062  2.14900997]
 [ 0.37530771  0.05705085  0.01716734  0.0189219   0.06967623  0.06471169]]
dE max: 4.65, min: 0.02, avg: 1.32
dE max: 0.00, min: 0.00, avg: 0.00
dE max: 0.00, min: 0.00, avg: 0.00


#### b. Acquire CC24 measurements from images

In [14]:
# Define essential parameters

wb_params = awb_params # (1.87217887201, 1.27358336204, 1.0, -16.2625453031, -13.099179932, 0.0) # or awb_params calculated before
repeats = 100

In [15]:
# Acquire preview image

DevList = mvsdk.CameraEnumerateDevice()
mycam = Camera(DevList[0], EXPOSURE_TIME, gain=1, hibitdepth=1)

mycam.open()
raw_img = mycam.grab_raw()
mycam.close()
# img最大值最小值
print(f'RAW Grabbed: img.max(): {raw_img.max()}, img.min(): {raw_img.min()}')
img = raw_img - BLC
img = raw_wb(img, 
             *wb_params, 
             pattern=BAYER_PATTERN, clip_max_level=ADC_MAX_LEVEL-BLC)
if BAYER_PATTERN == 'RGGB':
    img = cv2.cvtColor(img, cv2.COLOR_BAYER_RGGB2RGB)
elif BAYER_PATTERN == 'BGGR':
    img = cv2.cvtColor(img, cv2.COLOR_BAYER_BGGR2RGB)
print(f'After Debayer: img.max(): {img.max()}, img.min(): {img.min()}')
img = img.astype(np.float32) / 65535
plt.imshow(img)
plt.title('White balanced and debayered linear device RGB image')
plt.show()

Camera.open: Camera supports 2 pixel format(s)
Camera.open: Pixel format 0: Bayer BG 8bit (1Bpp)
Camera.open: Pixel format 1: Bayer BG 12bit Packed (1.5Bpp)
Camera.open: Using 12bit pixel format.
Camera.open: Timecode disabled. Allocating original image buffer size: 30081024 bytes.
Camera.open: Raw start bit = -1
RAW Grabbed: img.max(): 4094, img.min(): 32
After Debayer: img.max(): 65535, img.min(): 0


In [16]:
# Define 4 corners of the chart
chart_corners = np.array([
    [1126, 704],
    [1521, 704],
    [1126, 1294],
    [1521, 1294]
])

In [17]:
# Extract 24 patches using correction_utils

from correction_utils import calculate_warped_bboxes, extract_colors_warped_bbox, draw_warped_bboxes
# 1. Calculate the warped bboxes on the original image
# Since orientation is now locked to landscape, we use 4 rows and 6 columns.
warped_bboxes = calculate_warped_bboxes(chart_corners, rows=4, cols=6, margin_percent=0.2)

# 2. Extract mean colors directly from the original image `img`
device_rgb_values_sample = extract_colors_warped_bbox(img, warped_bboxes)

# 3. Visualize the bboxes
# Draw the warped bboxes on the original, uncorrected image
img_with_warped_bboxes = draw_warped_bboxes(img, warped_bboxes)

# 4. Print the values
for i, rgb in enumerate(device_rgb_values_sample):
    rgb_255 = tuple(f'{c * 255:.2f}' for c in rgb)
    print(f"Patch {i+1:2d}: Mean RGB = {rgb_255}")


plt.figure(figsize=(10, 8))
plt.imshow(img_with_warped_bboxes)
plt.title("Warped BBoxes on Original Image")
plt.axis('off')
plt.show()

Patch  1: Mean RGB = ('40.74', '24.78', '17.92')
Patch  2: Mean RGB = ('148.84', '96.26', '79.75')
Patch  3: Mean RGB = ('56.50', '72.92', '102.88')
Patch  4: Mean RGB = ('26.40', '30.54', '13.92')
Patch  5: Mean RGB = ('76.85', '79.26', '117.77')
Patch  6: Mean RGB = ('68.10', '114.91', '106.16')
Patch  7: Mean RGB = ('158.14', '63.71', '20.94')
Patch  8: Mean RGB = ('42.07', '59.12', '116.91')
Patch  9: Mean RGB = ('125.74', '42.64', '44.08')
Patch 10: Mean RGB = ('27.98', '23.60', '41.03')
Patch 11: Mean RGB = ('84.19', '96.70', '23.27')
Patch 12: Mean RGB = ('139.75', '75.19', '16.99')
Patch 13: Mean RGB = ('21.13', '34.66', '81.85')
Patch 14: Mean RGB = ('26.03', '55.88', '19.28')
Patch 15: Mean RGB = ('88.75', '22.22', '19.14')
Patch 16: Mean RGB = ('166.46', '111.93', '10.14')
Patch 17: Mean RGB = ('101.83', '47.31', '75.10')
Patch 18: Mean RGB = ('28.58', '62.25', '89.79')
Patch 19: Mean RGB = ('253.37', '249.41', '244.78')
Patch 20: Mean RGB = ('161.95', '162.16', '162.25')
Pa

In [18]:
# Acquire repeat measurements

from correction_utils import suppress_output
raw_wb_slience = suppress_output(raw_wb)

DevList = mvsdk.CameraEnumerateDevice()
mycam = Camera(DevList[0], EXPOSURE_TIME, gain=1, hibitdepth=1)
mycam.open()

wb_debayered_imgs = []
for i in range(repeats):
    raw_img = mycam.grab_raw()

    # WB + Debayer
    img = raw_img - BLC
    img = raw_wb_slience(img, 
                *wb_params, 
                pattern=BAYER_PATTERN, clip_max_level=ADC_MAX_LEVEL-BLC)
    if BAYER_PATTERN == 'RGGB':
        img = cv2.cvtColor(img, cv2.COLOR_BAYER_RGGB2RGB)
    elif BAYER_PATTERN == 'BGGR':
        img = cv2.cvtColor(img, cv2.COLOR_BAYER_BGGR2RGB)

    img = img.astype(np.float32) / 65535

    wb_debayered_imgs.append(img)
mycam.close()

Camera.open: Camera supports 2 pixel format(s)
Camera.open: Pixel format 0: Bayer BG 8bit (1Bpp)
Camera.open: Pixel format 1: Bayer BG 12bit Packed (1.5Bpp)
Camera.open: Using 12bit pixel format.
Camera.open: Timecode disabled. Allocating original image buffer size: 30081024 bytes.
Camera.open: Raw start bit = -1


In [19]:
# Or using measurements from awb run.
from correction_utils import suppress_output
raw_wb_slience = suppress_output(raw_wb)

wb_debayered_imgs = []
for raw_img in imgs:
    # WB + Debayer
    img = raw_img - BLC
    img = raw_wb_slience(img, 
                *wb_params, 
                pattern=BAYER_PATTERN, clip_max_level=ADC_MAX_LEVEL-BLC)
    if BAYER_PATTERN == 'RGGB':
        img = cv2.cvtColor(img, cv2.COLOR_BAYER_RGGB2RGB)
    elif BAYER_PATTERN == 'BGGR':
        img = cv2.cvtColor(img, cv2.COLOR_BAYER_BGGR2RGB)

    img = img.astype(np.float32) / 65535
    wb_debayered_imgs.append(img)

In [20]:
# Run color extraction from 24 patches
device_rgb_values_list = []
for img in wb_debayered_imgs:
    device_rgb_values = extract_colors_warped_bbox(img, warped_bboxes)
    device_rgb_values_list.append(device_rgb_values)
device_rgb_values_np = np.array(device_rgb_values_list)
print(f'device_rgb_values_np.shape: {device_rgb_values_np.shape}')
device_rgb_values = np.mean(device_rgb_values_np, axis=0)

device_rgb_values_np.shape: (100, 24, 3)


In [21]:
# Calculate correction matrix (so called Forward Matrix in DNG glossary)

fwd_mtx_identical = colour.characterisation.matrix_colour_correction_Cheung2004(device_rgb_values, XYZ_D65_values_identical)
fwd_mtx_bradford = colour.characterisation.matrix_colour_correction_Cheung2004(device_rgb_values, XYZ_D65_values_bradford)
fwd_mtx_cat02 = colour.characterisation.matrix_colour_correction_Cheung2004(device_rgb_values, XYZ_D65_values_cat02)

We could evaluate the quality of colour correction matrix by calculating $dE_{00}$ between corrected and original colours.

The corrected color should be carefully inversed to $Lab_{D50}$. \
Once the inversion is correct, the dE value should only be affected by the colour correction matrix.

In [22]:
# Calculate dE00 of nominal values and corrected values

CC24_XYZ_corrected_identical = device_rgb_values @ fwd_mtx_identical.T
CC24_XYZ_corrected_bradford = device_rgb_values @ fwd_mtx_bradford.T
CC24_XYZ_corrected_cat02 = device_rgb_values @ fwd_mtx_cat02.T

dE00_identical = colour.delta_E(lab_values_row_first, 
                      colour.XYZ_to_Lab(CC24_XYZ_corrected_identical, 
                                        illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65']
                                        )
                     )
dE00_bradford = colour.delta_E(lab_values_row_first, 
                      colour.XYZ_to_Lab(CC24_XYZ_corrected_bradford @ np.linalg.inv(color_correction_mtx_bradford).T, 
                                        illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']
                                        )
                     )
dE00_cat02 = colour.delta_E(lab_values_row_first, 
                      colour.XYZ_to_Lab(CC24_XYZ_corrected_cat02 @ np.linalg.inv(color_correction_mtx_cat02).T, 
                                        illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']
                                        )
                     )

print(f'dE00_identical: \n{dE00_identical.reshape(4,6)}')
print(f'  dE max: {np.max(dE00_identical):.2f}, min: {np.min(dE00_identical):.2f}, avg: {np.mean(dE00_identical):.2f}')
print(f'dE00_bradford: \n{dE00_bradford.reshape(4,6)}')
print(f'  dE max: {np.max(dE00_bradford):.2f}, min: {np.min(dE00_bradford):.2f}, avg: {np.mean(dE00_bradford):.2f}')
print(f'dE00_cat02: \n{dE00_cat02.reshape(4,6)}')
print(f'  dE max: {np.max(dE00_cat02):.2f}, min: {np.min(dE00_cat02):.2f}, avg: {np.mean(dE00_cat02):.2f}')


dE00_identical: 
[[ 1.91132224  3.53566835  3.24345017  0.74500562  0.62514335  1.97205114]
 [ 2.30899513  2.76655074  2.00306014  3.5179676   1.88586466  2.43833222]
 [ 4.50724413  1.69614996  2.80327743  1.18214713  4.32541145  4.1538634 ]
 [ 0.8538127   1.80441942  1.73861295  2.73804133  2.93468015  4.5865968 ]]
  dE max: 4.59, min: 0.63, avg: 2.51
dE00_bradford: 
[[ 1.91132224  3.53566835  3.24345017  0.74500562  0.62514335  1.97205114]
 [ 2.30899513  2.76655074  2.00306014  3.5179676   1.88586466  2.43833222]
 [ 4.50724413  1.69614996  2.80327743  1.18214713  4.32541145  4.1538634 ]
 [ 0.8538127   1.80441942  1.73861295  2.73804133  2.93468015  4.5865968 ]]
  dE max: 4.59, min: 0.63, avg: 2.51
dE00_cat02: 
[[ 1.91132224  3.53566835  3.24345017  0.74500562  0.62514335  1.97205114]
 [ 2.30899513  2.76655074  2.00306014  3.5179676   1.88586466  2.43833222]
 [ 4.50724413  1.69614996  2.80327743  1.18214713  4.32541145  4.1538634 ]
 [ 0.8538127   1.80441942  1.73861295  2.73804133  2.

In [23]:
# If the Lab value is not perfectly inverse, it may can reflect the actual performance
dE00_bradford_direct_lab = colour.delta_E(lab_values_row_first, 
                             colour.XYZ_to_Lab(CC24_XYZ_corrected_bradford, 
                               illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65']
                               )
                             )
dE00_cat02_direct_lab = colour.delta_E(lab_values_row_first, 
                             colour.XYZ_to_Lab(CC24_XYZ_corrected_cat02, 
                               illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65']
                               )
                             )
print('Direct inversion of Bradford to Lab at D65:')
print(f'  dE max: {np.max(dE00_bradford_direct_lab):.2f}, min: {np.min(dE00_bradford_direct_lab):.2f}, avg: {np.mean(dE00_bradford_direct_lab):.2f}')
print('Direct inversion of CAT02 to Lab at D65:')
print(f'  dE max: {np.max(dE00_cat02_direct_lab):.2f}, min: {np.min(dE00_cat02_direct_lab):.2f}, avg: {np.mean(dE00_cat02_direct_lab):.2f}')


Direct inversion of Bradford to Lab at D65:
  dE max: 5.81, min: 1.19, avg: 2.88
Direct inversion of CAT02 to Lab at D65:
  dE max: 5.74, min: 1.17, avg: 2.83


But the sRGB value **will** be affected by different chromatic adaptation methods.

In [24]:
# Calculate sRGB value in 8bit using different adaptation transforms
CC24_sRGB_corrected_identical = colour.XYZ_to_sRGB(CC24_XYZ_corrected_identical)
CC24_sRGB_corrected_bradford = colour.XYZ_to_sRGB(CC24_XYZ_corrected_bradford)

print(np.hstack(((CC24_sRGB_corrected_bradford * 255).astype(int), 
      (CC24_sRGB_corrected_identical * 255).astype(int)))
      )

[[ 118   83   66  120   83   66]
 [ 210  154  137  213  154  137]
 [  99  128  163   91  129  163]
 [  89  107   60   92  106   61]
 [ 128  125  173  122  126  172]
 [ 105  183  167  103  183  167]
 [ 231  129   45  236  129   45]
 [  60   93  174   25   95  173]
 [ 205   73   97  208   74   96]
 [  83   59  106   79   60  106]
 [ 158  189   75  166  188   77]
 [ 216  154   39  222  153   41]
 [ -23   57  148 -120   60  148]
 [  71  150   75   79  149   77]
 [ 179   38   56  182   39   55]
 [ 231  196  -27  240  195  -13]
 [ 177   68  135  176   70  135]
 [ -16  123  155  -89  124  155]
 [ 241  243  237  242  243  237]
 [ 197  200  197  197  200  197]
 [ 156  160  158  156  160  158]
 [ 113  115  113  113  115  114]
 [  74   77   76   74   77   77]
 [  37   42   40   37   42   40]]


In [25]:
# Validate correction by performing a forward transformation to sRGB
img_XYZ_corrected_identical = img @ fwd_mtx_identical.T
img_XYZ_corrected_bradford = img @ fwd_mtx_bradford.T
img_XYZ_corrected_cat02 = img @ fwd_mtx_cat02.T

img_sRGB_identical = colour.XYZ_to_sRGB(img_XYZ_corrected_identical, chromatic_adaptation_transform='Bradford', apply_cctf_encoding=True)
img_sRGB_bradford = colour.XYZ_to_sRGB(img_XYZ_corrected_bradford, chromatic_adaptation_transform='Bradford', apply_cctf_encoding=True)
img_sRGB_cat02 = colour.XYZ_to_sRGB(img_XYZ_corrected_cat02, chromatic_adaptation_transform='Bradford', apply_cctf_encoding=True)
# The chromatic_adaptation_transform should not affect anything since the sRGB is D65 based.

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 5))
ax1.imshow(img)
ax1.set_title('Original Image (linear, device RGB)')
ax1.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax1.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax2.imshow(img_sRGB_identical)
ax2.set_title('Corrected Image (Implicit Identical)')
ax2.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax2.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax3.imshow(img_sRGB_bradford)
ax3.set_title('Corrected Image (Bradford)')
ax3.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax3.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax4.imshow(img_sRGB_cat02)
ax4.set_title('Corrected Image (CAT02)')
ax4.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax4.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.14944240805..1.12595000329].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.73510751741..1.09827472641].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-1.72443898785..1.09761140957].


#### b2. Conclusion
Visually, the CAT02 method is the best.

We visualize the whole process of RAW development here.

In [26]:
# Alternative efficient calculation (using CAT02 as example) showing the whole process:

from correction_utils import linear_to_srgb

device_RGB_to_linear_sRGB = colour.RGB_COLOURSPACES['sRGB'].matrix_XYZ_to_RGB @ fwd_mtx_cat02
img_sRGB_cat02_linear = np.clip(img @ device_RGB_to_linear_sRGB.T, 0, 1)
img_sRGB_cat02 = linear_to_srgb(img_sRGB_cat02_linear)

fig, ((ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2, 4, figsize=(16, 10))
ax1.imshow(raw_img)
ax1.set_title("RAW Image")
ax1.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax1.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax2.imshow(img)
ax2.set_title("WB+Debayered Image (Linear)")
ax2.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax2.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax3.imshow(img_sRGB_cat02_linear)
ax3.set_title("CAT02+Cheung2004 Fwd Corr. Img. (Linear)")
ax3.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax3.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax4.imshow(img_sRGB_cat02)
ax4.set_title("CAT02+Cheung2004 Fwd Corr. Img. (sRGB)")
ax4.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax4.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax5.imshow(cv2.cvtColor(raw_img, cv2.COLOR_BAYER_RG2RGB)/4096)
ax5.set_title('Debayer Only (Device RGB)')
ax5.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax5.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax6.imshow(linear_to_srgb(img))
ax6.set_title('WB+Debayered in sRGB gamma')
ax6.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax6.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
plt.tight_layout()
plt.show()

#### b3. Optimize calculation process to accelerate the transformation.

In [27]:
# TODO: Maybe a C extension.
# Implemented 250731 13ms/2448x2048@12bit, uint16

#### c. Using customized loss function (dE2000 based) to calculate forward matrix.

In [28]:
# A good initial value is essential for better dE.

sigma_init = 0.01

from correction_utils import forward_matrix_slover

fwd_mtx_custom_solver, final_loss = forward_matrix_slover(device_rgb_values, XYZ_D50_values, 
                                                   illuminant='D50',
                                                   initial_matrix=fwd_mtx_cat02+np.random.normal(0, sigma_init, size=(3,3)),
                                                   extra_illuminant_transform_XYZ=np.linalg.inv(color_correction_mtx_cat02),
                                                   )


Iteration  100, Loss (dE2000**3): 19.9681
Iteration  200, Loss (dE2000**3): 19.2230
Iteration  300, Loss (dE2000**3): 19.0495
Iteration  400, Loss (dE2000**3): 19.0390
Iteration  500, Loss (dE2000**3): 19.0228
Iteration  600, Loss (dE2000**3): 18.8168
Iteration  700, Loss (dE2000**3): 18.7198
Iteration  800, Loss (dE2000**3): 18.7105
Iteration  900, Loss (dE2000**3): 18.7082
Iteration 1000, Loss (dE2000**3): 18.7082

Optimization finished.
Final Loss (dE2000**3): 18.7082


#### d. Compare the results and choose which one to save.

In [30]:
# Print and render results
print('custom forward_matrix: ')
print(fwd_mtx_custom_solver)

print('fwd_mtx by colour-science: ')
print(fwd_mtx_cat02)

CC24_XYZ_corrected_custom_fwd = device_rgb_values @ fwd_mtx_custom_solver.T

dE00_custom_fwd_direct_lab = colour.delta_E(lab_values_row_first, 
                               colour.XYZ_to_Lab(CC24_XYZ_corrected_custom_fwd @ np.linalg.inv(color_correction_mtx_cat02).T, 
                                 illuminant=colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']
                                 )
                               )

print('dE00 from custom forward matrix:')
print(dE00_custom_fwd_direct_lab)
print(f'  dE max: {np.max(dE00_custom_fwd_direct_lab):.2f}, min: {np.min(dE00_custom_fwd_direct_lab):.2f}, avg: {np.mean(dE00_custom_fwd_direct_lab):.2f}')
print('dE00 from CAT02:')
print(dE00_cat02_direct_lab)
print(f'  dE max: {np.max(dE00_cat02_direct_lab):.2f}, min: {np.min(dE00_cat02_direct_lab):.2f}, avg: {np.mean(dE00_cat02_direct_lab):.2f}')

img_sRGB_custom_fwd = linear_to_srgb(np.clip(img @ fwd_mtx_custom_solver.T @ colour.RGB_COLOURSPACES['sRGB'].matrix_XYZ_to_RGB.T, 0, 1))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(img_sRGB_custom_fwd)
ax1.set_title("Custom Forward Matrix")
ax1.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax1.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax1.axis('off')
ax2.imshow(img_sRGB_cat02)
ax2.set_title("CAT02 Forward Matrix")
ax2.set_xlim(chart_corners[:, 0].min(), chart_corners[:, 0].max())
ax2.set_ylim(chart_corners[:, 1].max(), chart_corners[:, 1].min())
ax2.axis('off')
plt.show()

custom forward_matrix: 
[[ 0.53777609  0.39176303 -0.07823343]
 [ 0.15362751  1.06443876 -0.31712604]
 [-0.09106248  0.23445596  0.81987655]]
fwd_mtx by colour-science: 
[[ 0.48400457  0.49366056 -0.12875313]
 [ 0.11755779  1.14741225 -0.35974064]
 [-0.10007283  0.26020384  0.80710571]]
dE00 from custom forward matrix:
[ 1.74179363  3.89205456  3.57741507  1.29240565  1.25222719  1.87653517
  2.48320028  2.0703755   2.68013947  2.28412688  2.19893733  2.49734834
  2.36746242  0.9990171   2.18582551  1.46885639  2.89822994  4.21043201
  0.98819987  0.87282769  1.15580178  2.43619591  2.6476921   4.37289933]
  dE max: 4.37, min: 0.87, avg: 2.27
dE00 from CAT02:
[ 2.24446063  3.33668851  4.44900076  1.8306594   1.89308657  2.06439773
  2.33215484  2.66527507  2.10619767  2.08273177  3.30988122  3.72550504
  2.44595691  1.47184807  3.37156307  3.29109667  4.39005257  5.73675453
  1.16693104  1.90675781  1.76869839  2.78378092  2.92742359  4.62938848]
  dE max: 5.74, min: 1.17, avg: 2.83


In [30]:
# Visualize the bases of the device RGB space in XYZ color model.

# Note on linear algebra: the conversion matrix which map points from space A to B, 
# literally equals to horizontal stacks of bases of space A in the coordinates of space B.
# (Here, we assume all vectors are vertically arranged.)

fig, ax = colour.plotting.plot_RGB_colourspaces_gamuts('sRGB', 'CIE XYZ')
ax.scatter(fwd_mtx_custom_solver[:, 0], fwd_mtx_custom_solver[:, 1], fwd_mtx_custom_solver[:, 2], c = ('red', 'green', 'blue'))
fig.show()

#### e. Choose and save correction results in numpy file.

In [31]:
# Save tag: BLC, ADC_MAX_LEVEL, wb_params, fwd_mtx and Notes

Notes = 'D50 15% power, vertical illuminant, 10ms exposure, Analog Gain 1.0, 8mm Lens, IMX264, 5MP'
mtx_to_save = fwd_mtx_cat02
filename = 'correction_results_D65_250731.npy'

np.save(filename, {'BLC': BLC, 'ADC_MAX_LEVEL': ADC_MAX_LEVEL,
                                   'wb_params': wb_params, 'fwd_mtx': mtx_to_save, 
                                   'Notes': Notes
                                   })

# Read back and check
data = np.load(filename, allow_pickle=True).item()
print(data)

{'BLC': 32, 'ADC_MAX_LEVEL': 4095, 'wb_params': (1.8538751110194334, 1.2618050211142049, 1.0, -14.453442776099127, -11.818613731445266, 0.0), 'fwd_mtx': array([[ 0.48400457,  0.49366056, -0.12875313],
       [ 0.11755779,  1.14741225, -0.35974064],
       [-0.10007283,  0.26020384,  0.80710571]]), 'Notes': 'D50 15% power, vertical illuminant, 10ms exposure, Analog Gain 1.0, 8mm Lens, IMX264, 5MP'}


`wb_params`: (R_gain, G_gain, B_gain, R_dBLC, G_dBLC, B_dBLC)

`fwd_mtx`: Transform from device RGB to $XYZ$ in given illuminant (e.g. D50). 
The user should calculate chromatic (illuminant) adaptation and $XYZ$->sRGB itself.
如果已经wb，是否还需要改变fwd_mtx? 可能有微调。测试一下。

In [33]:
data1 = np.load('correction_results_D65_250731.npy', allow_pickle=True).item()
data2 = np.load('correction_results_D50_250731.npy', allow_pickle=True).item()
print(data1['wb_params'])
print(data2['wb_params'])
print(data1['fwd_mtx'])
print(data2['fwd_mtx'])


(1.8538751110194334, 1.2618050211142049, 1.0, -14.453442776099127, -11.818613731445266, 0.0)
(1.6232238624261359, 1.3050488646488514, 1.0, -6.6773836516730469, -8.8718066707359053, 0.0)
[[ 0.48400457  0.49366056 -0.12875313]
 [ 0.11755779  1.14741225 -0.35974064]
 [-0.10007283  0.26020384  0.80710571]]
[[ 0.51162398  0.57441403 -0.13854522]
 [ 0.15561742  1.30252441 -0.44838334]
 [-0.09894102  0.17512108  1.0111746 ]]
